In [57]:
import math
import pandas as pd
import numpy as np

In [58]:
# importing inaugural corpus from nltk and using as document
#query string

import nltk
nltk.download('inaugural')
from nltk.corpus import inaugural
doc1 = " ".join(inaugural.words('1865-Lincoln.txt'))
doc2 = " ".join(inaugural.words('1933-Roosevelt.txt'))
doc3 = " ".join(inaugural.words('2009-Obama.txt'))

query = "life learning"


#print(doc3)

[nltk_data] Downloading package inaugural to
[nltk_data]     C:\Users\nusan\AppData\Roaming\nltk_data...
[nltk_data]   Package inaugural is already up-to-date!


In [59]:
#finding term frequency from documents
#term -frequenvy :word occurences in a document

def compute_tf(docs_list):
    for doc in docs_list: 
        doc1_lst = doc.split(" ")
        wordDict_1= dict.fromkeys(set(doc1_lst), 0)

        for token in doc1_lst:
            wordDict_1[token] +=  1
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    #adding new column for term frequency count
        df.insert(loc=idx, column='Document', value=new_col) #adding new column for document name
        print(df)
        
compute_tf([doc1, doc2, doc3])

         Document  achieve  could  ascribe  astounding  just  impending  read  \
0  Term Frequency        1      2        1           1     2          1     1   

   knew  Countrymen  ...  shall  is  whole  gives  by  nations  ?  a  wealth  \
0     1           1  ...      5   6      1      2   6        1  1  7       1   

   wounds  
0       1  

[1 rows x 364 columns]
         Document  emergency  yours  grim  situation  tragedy  policy  rights  \
0  Term Frequency          4      1     1          1        1       3       1   

   products  Congress  ...  selfish  purchase  by  candor  Hand  ways   a  \
0         1         4  ...        1         1  19       1     1     1  38   

   wealth  can  midst  
0       1   11      1  

[1 rows x 747 columns]
         Document  humble  achieve  celebrated  rights  fist  much  hard  \
0  Term Frequency       1        1           1       1     1     1     4   

   faithful  free  ...  sacrifices  energy  ways   a  wealth  midst  can  far  \
0   

In [60]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split() #normalizing using lowering the words
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf= dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    #adding new col for normalized term frequency count
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
    return tf_doc
documents = [doc1, doc2, doc3]

tf_doc = compute_normalizedtf([doc1, doc2, doc3])

        Document   achieve     could   ascribe  astounding      just  \
0  Normalized TF  0.001274  0.002548  0.001274    0.001274  0.002548   

   impending      read      knew  Countrymen  ...     shall        is  \
0   0.001274  0.001274  0.001274    0.001274  ...  0.006369  0.007643   

      whole     gives        by   nations         ?         a    wealth  \
0  0.001274  0.002548  0.007643  0.001274  0.001274  0.008917  0.001274   

     wounds  
0  0.001274  

[1 rows x 364 columns]
        Document  emergency     yours      grim  situation   tragedy  \
0  Normalized TF   0.001939  0.000485  0.000485   0.000485  0.000485   

     policy    rights  products  Congress  ...   selfish  purchase       by  \
0  0.001454  0.000485  0.000485  0.001939  ...  0.000485  0.000485  0.00921   

     candor      Hand      ways        a    wealth       can     midst  
0  0.000485  0.001939  0.000485  0.01842  0.000485  0.005332  0.000485  

[1 rows x 747 columns]
        Document    humble   ac

In [61]:
#Normalized TF for the query string("life learning")
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

{'life': 0.5, 'learning': 0.5}


In [69]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
#Create a variation of Program_1 that uses only TF representation to compute the similarity between a query and document.  
def cosine_similarity(tf_doc, query, doc_num):
     dot_product = 0
     qry_mod = 0
     doc_mod = 0
     tokens = query.split()
     
     for keyword in tokens:
          dot_product += query_norm_tf[keyword] * tf_doc[keyword][tf_doc['Document'] == doc_num]
          qry_mod += (query_norm_tf[keyword] * query_norm_tf[keyword])
          doc_mod += (tf_doc[keyword][tf_doc['Document'] == doc_num] * tf_doc[keyword][tf_doc['Document'] == doc_num])
     qry_mod = math.sqrt(qry_mod)
     doc_mod = math.sqrt(doc_mod)
     
     #implement formula
     denominator = qry_mod * doc_mod
     cos_sim = dot_product / denominator
     return cos_sim


from collections import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                yield x
        else:        
             yield item
     

In [72]:
def rank_similarity_docs(data):
    cos_sim = []
    for doc_num in range(len(data)):
        cos_sim.append(cosine_similarity(tf_doc, query, doc_num).tolist())
    return cos_sim
similarity_docs = rank_similarity_docs(tf_doc)
doc_names = ["1865-Lincoln.txt", "1933-Roosevelt.txt", "2009-Obama.txt"]
print("Similarity between query and documents")
print(doc_names)
print(list(flatten(similarity_docs)))

TypeError: list indices must be integers or slices, not str